In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = "/content/drive/MyDrive/Colab_Notebooks/NLP/Assignment\ 7/datasets/"
!cp -R $data .

# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [3]:
#langchain library
!pip3 install langchain==0.1.0
#LLM
!pip3 install accelerate==0.25.0
!pip3 install transformers==4.36.2
!pip3 install bitsandbytes==0.41.2
#Text Embedding
!pip3 install sentence-transformers==2.2.2
!pip3 install InstructorEmbedding==1.0.1
#vectorstore
!pip3 install pymupdf==1.23.8
!pip3 install faiss-gpu==1.7.2
!pip3 install faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [6]:
from langchain import PromptTemplate

# prompt_template = """
#     I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).
#     If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.
#     Whether it's about probabilistic models, language models, or any other related topic,
#     I'm here to help break down complex concepts into easy-to-understand explanations.
#     Just let me know what you're wondering about, and I'll do my best to guide you through it!
#     {context}
#     Question: {question}
#     Answer:
#     """.strip()

# AIT GPT Template
prompt_template = """
        Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).
        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.
        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.
        {context}
        Query: {question}
        Answer:
        """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).\n        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.\n        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.\n        {context}\n        Query: {question}\n        Answer:")

In [7]:
PROMPT.format(
    context = "The Asian Institute of Technology (AIT) is an international English-speaking postgraduate institution, focusing on engineering, environment, and management studies. AIT’s rigorous academic, research, and experiential outreach programs prepare graduates for professional success and leadership roles in Asia and beyond.",
    question = "What is AIT"
)

"Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).\n        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.\n        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.\n        The Asian Institute of Technology (AIT) is an international English-speaking postgraduate institution, focusing on engineering, environment, and management studies. AIT’s rigorous academic, research, and experiential outreach programs prepare graduates for professional success and leadership roles in Asia and beyond.\n        Query: What is AIT\n        Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [8]:
# from langchain.document_loaders import PyMuPDFLoader

# nlp_docs = 'datasets/AEA_Event_Brochure.pdf'

# loader = PyMuPDFLoader(nlp_docs)
# documents = loader.load()

In [9]:
# multiple pdf

import os
from langchain.document_loaders import PyMuPDFLoader

# Path to the folder containing PDF documents
folder_path = 'datasets'

# Initialize an empty list to store loaded documents
documents = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    # Check if the file is a PDF
    if file_name.endswith('.pdf'):
        # Construct the full path to the PDF file
        pdf_path = os.path.join(folder_path, file_name)

        # Load the PDF document
        loader = PyMuPDFLoader(pdf_path)
        document = loader.load()

        # Add the loaded documents to the list
        documents.extend(document)

# Now, all_documents contains the loaded documents from all PDFs in the "datasets" folder

In [10]:
# documents

In [11]:
len(documents)

405

In [12]:
documents[1]

Document(page_content='COST\nThe\nThe Flexible Degree Options has the same cost as our regular \n“On-Campus Option”. However, all Schools provide AIT \nScholarships for eligible candidates. To give you more ﬂexibility, \nwe allow you to pay course-by-course. You need to register for a \nminimum of 6 credits per semester (equivalent to 2 courses). You \nmay take up to 15 credits per semester. Some additional fees \nsuch as an initial application fee and Language Center fee are \nwaived.\nwaived.\nPayment\nFirst, you need to decide for how many courses you want to \nenrollee, the minimum enrollment is 6 credits which is equivalent \nto 2 courses. The Program Secretary can advise you on details. \nYou need to pay the full fee for your courses before the start of \nthe semester.\nDETAILS\nFor  Masters  www.ait.ac.th/programs/ﬂexible-masters-option/\nFor  PhD  www.ait.ac.th/programs/new-ﬂexible-phd-option/\nCONTACT\nFor School of Engineering and \nTechnology: fdoset@ait.ac.th\nFor School of

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [14]:
doc[1]

Document(page_content='• Data Science and AI\n• Remote Sensing and Geographic Information System\nDEPARTMENT OF FOOD, AGRICULTURE AND BIORESOURCES\n•   Agribusiness Management (ABM)\n•   Agricultural Systems and Engineering (ASE)\n•   Food Innovation, Nutrition and Health (FINH)\nDEPARTMENT OF DEVELOPMENT AND SUSTAINABILITY\n• Natural Resources Management (NRM)\n• D\n• Development Planning Management and Innovation (DPMI)\n• Society and Environmental Governance (SEG)\nDEPARTMENT OF ENERGY, ENVIRONMENT AND CLIMATE \nCHANGE\n• Climate Change and Sustainable Development (CCSD)\nSCHOO;L OF MANAGEMENT\n• Business Administration (MBA)\n• • Business Analytics and Digital Transformation\n• International Finance', metadata={'source': 'datasets/Flexible-Master-Option-2021.pdf', 'file_path': 'datasets/Flexible-Master-Option-2021.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe Photoshop CC 2019 (Macintosh)', 'produ

In [15]:
len(doc)

1721

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [16]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [17]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [18]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [19]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [20]:
#ready to use
retriever = vectordb.as_retriever()

In [21]:
retriever.get_relevant_documents("What is Dependency Parsing")

[Document(page_content='Because Web services are emerged to serve the requirement of independent languages and independent environments, the \ncore supporting technologies consist of the Extensible Markup Language (XML) for sharing data, the Simple Object Access \nProtocol (SOAP) for interacting with Web services, the Universal Discovery, Description and Integration (UDDI) for filling the \ndiscovery and description need, and the Web Services Description Language (WSDL) for defining services all of them are \nopen, cross-platform standards. \n \nThis proposed methodology covers overall life cycle of software development. In analysis and design phases, object-oriented', metadata={'source': 'datasets/CSIM-Thesis-2001.pdf', 'file_path': 'datasets/CSIM-Thesis-2001.pdf', 'page': 2, 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'Computer Science', 'author': 'Amy', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 6.0 for Word', 'producer': 'Acrobat Distiller 6.0 (Windows)', 'crea

In [22]:
retriever.get_relevant_documents("What is Transformers")

[Document(page_content='Earthquake Disaster in Mexico City (1985)\nMuch of the destruction was due to significant amplification \nof EQ ground motions by thick soft soil deposits.', metadata={'source': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'file_path': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'page': 14, 'total_pages': 69, 'format': 'PDF 1.5', 'title': 'Slide 1', 'author': 'Pennunng', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20140813162040+07'00'", 'modDate': "D:20140813162040+07'00'", 'trapped': ''}),
 Document(page_content='A rigid model fixed on the multi-component force sensor\nWind\nFx (Drag force)\nFz (Vertical force)\nMy (Overturning moment)\nMz  (Torsion)\nMx\n(Transverse moment )\nFy (Side force)\nHigh Frequency Force Balance Technique', metadata={'source': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'file_path': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'page': 54, 'total_pages': 

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [23]:
!pip3 install flask-sqlalchemy

In [24]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [25]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [26]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [28]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [29]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [30]:
# !rm -rf models

In [31]:
%mkdir models
%cd ./models
# os.environ["TOKENIZERS_PARALLELISM"] = 'false'
!git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

/content/models
Cloning into 'fastchat-t5-3b-v1.0'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), 8.84 KiB | 532.00 KiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model.bin

See: `git lfs help smudge` for more details.


In [32]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = '../models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [33]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [34]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [35]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [36]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [37]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).\n        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.\n        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.\n        {context}\n        Query: {question}\n        Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7ab62a7e8ac0>)), document_variable_name='context')

In [38]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).
        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.
        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.
        A rigid model fixed on the multi-component force sensor
Wind
Fx (Drag force)
Fz (Vertical force)
My (Overturning moment)
Mz  (Torsion)
Mx
(Transverse moment )
Fy (Side force)
High Frequency Force Balance Technique

Earthquake Disaster in Mexico City (1985)
Much of the destruction was due to significant amplification 
of EQ ground motions by thick soft soil deposits.

Mechatronics, vol. 7, 

{'input_documents': [Document(page_content='A rigid model fixed on the multi-component force sensor\nWind\nFx (Drag force)\nFz (Vertical force)\nMy (Overturning moment)\nMz  (Torsion)\nMx\n(Transverse moment )\nFy (Side force)\nHigh Frequency Force Balance Technique', metadata={'source': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'file_path': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'page': 54, 'total_pages': 69, 'format': 'PDF 1.5', 'title': 'Slide 1', 'author': 'Pennunng', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20140813162040+07'00'", 'modDate': "D:20140813162040+07'00'", 'trapped': ''}),
  Document(page_content='Earthquake Disaster in Mexico City (1985)\nMuch of the destruction was due to significant amplification \nof EQ ground motions by thick soft soil deposits.', metadata={'source': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'file_path': 'datasets/AITC_CSI_Wind_EQ_Overview.pdf', 'page'

In [39]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).\n        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.\n        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.\n        {context}\n        Query: {question}\n        Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7ab6

## 5. Chatbot

In [40]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).
        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.
        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.
        PhD, Kent State University, USA 
Associate 
Professor 
(Electronic 
Commerce/Electronic 
Business, 
Web-based Information Systems, 
Hypermedia, 
Electronic 
Government) 
MATTHEW N. DAILEY, BSc, MSc, 
North Carolina State University, PhD, 
University of California, San Diego. 
Assistant Professor (Machine Vision 
& 
Learning, 
Robotics,

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': "<pad>  I'm  an  AI  language  model  and  do  not  have  personal  experiences.  I  am  here  to  provide  information  and  insights  about  the  Asian  Institute  of  Technology  (AIT).  Is  there  anything  specific  you  would  like  to  know  about  AIT?\n",
 'source_documents': [Document(page_content='PhD, Kent State University, USA \nAssociate \nProfessor \n(Electronic \nCommerce/Electronic \nBusiness, \nWeb-based Information Systems, \nHypermedia, \nElectronic \nGovernment) \nMATTHEW N. DAILEY, BSc, MSc, \nNorth Carolina State University, PhD, \nUniversity of California, San Diego. \nAssistant Professor (Machine Vision \n& \nLearning, \nRobotics, \nSoftware \nEngineering \nand \nOpen-source \nSoftware Development) \nPAUL JANECEK, BSEE, US Military \nAcademy; \nMSc, \nUniversity \nof \nLondon, UK; PhD, Swiss Federal \nInstitute of Technology, Switzerland. \nAssistant \nProfessor \n(Human-\nComputer Interact

In [41]:
prompt_question = "What is the Transformers?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I'm  an  AI  language  model  and  do  not  have  personal  experiences.  I  am  here  to  provide  information  and  insights  about  the  Asian  Institute  of  Technology  (AIT).  Is  there  anything  specific  you  would  like  to  know  about  AIT?\n")]
Follow Up Input: What is the Transformers?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).
        Whether you're a prospective student, a current stu

{'question': 'What is the Transformers?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad>  I'm  an  AI  language  model  and  do  not  have  personal  experiences.  I  am  here  to  provide  information  and  insights  about  the  Asian  Institute  of  Technology  (AIT).  Is  there  anything  specific  you  would  like  to  know  about  AIT?\n")],
 'answer': '<pad>  Transformers  are  electronic  devices  that  convert  electrical  energy  into  a  different  form  of  energy.  They  are  used  in  a  wide  range  of  applications,  including  power  electronics,  communication  systems,  and  industrial  control  systems.  Transformers  are  used  to  convert  AC  power  into  DC  power,  to  convert  DC  power  into  AC  power,  and  to  convert  AC  power  into  DC  power.  They  are  also  used  to  control  the  flow  of  current  in  electrical  circuits.\n',
 'source_documents': [Document(page_content='A rigid model fixed on the mult

In [42]:
prompt_question = "Explain about AIT?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I'm  an  AI  language  model  and  do  not  have  personal  experiences.  I  am  here  to  provide  information  and  insights  about  the  Asian  Institute  of  Technology  (AIT).  Is  there  anything  specific  you  would  like  to  know  about  AIT?\n"), HumanMessage(content='What is the Transformers?'), AIMessage(content='<pad>  Transformers  are  electronic  devices  that  convert  electrical  energy  into  a  different  form  of  energy.  They  are  used  in  a  wide  range  of  applications,  including  power  electronics,  communication  systems,  and  industrial  control  systems.  Transformers  are  used  to  convert  AC  power  i

{'question': 'Explain about AIT?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad>  I'm  an  AI  language  model  and  do  not  have  personal  experiences.  I  am  here  to  provide  information  and  insights  about  the  Asian  Institute  of  Technology  (AIT).  Is  there  anything  specific  you  would  like  to  know  about  AIT?\n"),
  HumanMessage(content='What is the Transformers?'),
  AIMessage(content='<pad>  Transformers  are  electronic  devices  that  convert  electrical  energy  into  a  different  form  of  energy.  They  are  used  in  a  wide  range  of  applications,  including  power  electronics,  communication  systems,  and  industrial  control  systems.  Transformers  are  used  to  convert  AC  power  into  DC  power,  to  convert  DC  power  into  AC  power,  and  to  convert  AC  power  into  DC  power.  They  are  also  used  to  control  the  flow  of  current  in  electrical  circuits.\n')],
 'answer': "<pad> < 

In [43]:
answer['answer']

"<pad> < pad>  The  Asian  Institute  of  Technology  (AIT)  is  an  international  institute  of  higher  learning  established  in  1959  to  help  meet  the  region's  growing  needs  for  advanced  learning  in  engineering,  science,  technology,  and  management,  research,  and  capacity  building.  AIT's  mission  is  to  develop  highly  qualified  and  committed  professionals  who  will  play  a  leading  role  in  the  sustainable  development  of  the  region  and  its  integration  into  the  global  economy.  AIT  is  based  in  Thailand  and  has  affiliated  centers  in  other  parts  of  the  world.\n"

In [44]:
# !mkdir models

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [53]:
import cloudpickle

file_path = '/content/models/chatbots.pkl'
with open(file_path, 'wb') as f:
  cloudpickle.dump(chain, f)

In [70]:
def load_model():
    with open(file_path, 'rb') as f:
        chain = cloudpickle.load(f)
    return chain

model = load_model()

In [71]:
prompt_question = "Explain about AIT?"
answer = model({"question":prompt_question})


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I'm your knowledgeable AIT assistant, here to provide you with insights and information about the Asian Institute of Technology (AIT).
        Whether you're a prospective student, a current student, or simply curious about AIT, feel free to ask anything related to the university, and I'll do my best to provide you with clear and informative answers.
        From admission procedures to academic programs, campus life, and more, I'm here to assist you in exploring all facets of AIT.
        ASIAN INSTITUTE OF TEHCNOLOGY
THAILAND
The Asian Institute of Technology (AIT) is an international institute of higher learning. It is Asia’s 
pioneer institution established in 1959 to help meet region’s   growing needs for advanced learning 
in engineering, science, technology and management, research and capacity building. AIT’s 
mission is to develop highly qualified

In [72]:
answer['answer']

"<pad> < pad>  The  Asian  Institute  of  Technology  (AIT)  is  an  international  institute  of  higher  learning  established  in  1959  to  help  meet  the  region's  growing  needs  for  advanced  learning  in  engineering,  science,  technology,  and  management,  research  and  capacity  building.  AIT's  mission  is  to  develop  highly  qualified  and  committed  professionals  who  will  play  a  leading  role  in  the  sustainable  development  of  the  region  and  its  integration  into  the  global  economy.  AIT  is  based  in  Thailand  and  has  affiliated  centers  in  other  parts  of  the  world.\n"

In [68]:
dest = "/content/drive/MyDrive/Colab_Notebooks/NLP/Assignment\ 7/models/"

In [55]:
file_path

'/content/aaa/chatbots.pkl'

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [69]:
!cp $file_path $dest